# Projeto: Impacto do Airbnb em Fatores Urbanos em Portugal (REVER AINDA)
Este notebook faz parte da Fase 1 do projeto da unidade curricular.



## Objetivos:
- Integrar dados de diferentes fontes (Airbnb, INE, etc.)
- Analisar a relação entre densidade de alojamentos locais, população e rendas
- Implementar técnicas de data cleaning, schema integration e identity resolution


In [36]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Leitura dos datasets
# O dataset listings contém informações sobre os imóveis disponíveis para venda
listings = pd.read_csv(r'dados\portugal_listings.csv')

# O dataset rendas contém informações sobre os preços das rendas por metro quadrado
rendas = pd.read_json(r'dados\rendasm2.json')
rendas = pd.DataFrame(rendas['Dados'][0]['2023'])

# O dataset densidade_pop contém informações sobre a densidade populacional (Nº/km2) por conselho
densidade_pop = pd.read_json(r'dados\densidadePopulacional.json')
densidade_pop = pd.DataFrame(densidade_pop['Dados'][0]['2023'])

# O dataset densidade_aloj contém informações sobre a densidade de alojamentos (Nº/km2) por freguesia
densidade_aloj = pd.read_json(r'dados\densidadealojamentosm2.json')
densidade_aloj = pd.DataFrame(densidade_aloj['Dados'][0]['2021'])

# A tabela municipios contém informações sobre a ordenação territorial portuguesa
concelhos = pd.read_excel(r'dados\Areas_Freg_Conc_Dist_Pais_CAOP2019.xls', sheet_name='Areas_Concelhos_CAOP2019')

In [37]:

import unidecode

# Função para normalizar texto
def normalizar(texto):
    return unidecode.unidecode(str(texto).strip().lower())

# Manter apenas as colunas relevantes
colunasParaManter = ['DICO', 'CONCELHO_DSG', 'NUTSIII_DSG']
concelhos = concelhos[colunasParaManter]

# Renomear colunas
concelhos = concelhos.rename(columns={
    'DICO': 'MunicipalityCode',
    'CONCELHO_DSG': 'Municipality',
    'NUTSIII_DSG': 'Region'
})

# Normalizar e preparar chaves
concelhos['MunicipalityCode'] = concelhos['MunicipalityCode'].astype(str).str.zfill(4)
concelhos['MunicipalityNorm'] = concelhos['Municipality'].apply(normalizar)

# --- Densidade Populacional ---
dfPop = pd.read_json('dados/DensidadePopulacional.json')['Dados'][0]['2023']
dfPop = pd.DataFrame(dfPop)
dfPop['MunicipalityCode'] = dfPop['geocod'].str[-4:]
dfPop['MunicipalityNorm'] = dfPop['geodsg'].apply(normalizar)

dfPopMerged = dfPop.merge(concelhos, on='MunicipalityCode', how='left', suffixes=('', '_ref'))

# --- Densidade Alojamentos ---
dfAloj = pd.read_json('dados/densidadealojamentosm2.json')['Dados'][0]['2021']
dfAloj = pd.DataFrame(dfAloj)
dfAloj['MunicipalityCode'] = dfAloj['geocod'].str[:4]
dfAloj['MunicipalityNorm'] = dfAloj['geodsg'].apply(normalizar)

dfAlojMerged = dfAloj.merge(concelhos, on='MunicipalityCode', how='left', suffixes=('', '_ref'))

# --- Filtrar matches válidos ---
populacionalMatch = dfPopMerged[dfPopMerged['Municipality'].notna()].copy()
alojamentosMatch = dfAlojMerged[dfAlojMerged['Municipality'].notna()].copy()

# --- Converter densidades para float ---
populacionalMatch['Density'] = pd.to_numeric(populacionalMatch['valor'], errors='coerce')
alojamentosMatch['Density'] = pd.to_numeric(alojamentosMatch['valor'], errors='coerce')

# --- Agregar médias por concelho ---
densidadePopMedia = (
    populacionalMatch
    .groupby('MunicipalityCode')
    .agg({'Density': 'mean'})
    .reset_index()
    .rename(columns={'Density': 'DensityPopMean'})
)

densidadeAlojMedia = (
    alojamentosMatch
    .groupby('MunicipalityCode')
    .agg({'Density': 'mean'})
    .reset_index()
    .rename(columns={'Density': 'DensityAlojMean'})
)

# --- Juntar nomes e regiões ---
densidadePopMedia = densidadePopMedia.merge(
    concelhos[['MunicipalityCode', 'Municipality', 'Region']], on='MunicipalityCode', how='left'
)

densidadeAlojMedia = densidadeAlojMedia.merge(
    concelhos[['MunicipalityCode', 'Municipality', 'Region']], on='MunicipalityCode', how='left'
)

# --- Exportar ---
densidadePopMedia.to_csv('densidadePopTratada.csv', index=False)
densidadeAlojMedia.to_csv('densidadeAlojTratada.csv', index=False)

# --- Exportar tabela dos concelhos (normalizada) ---
dfConcelhosTratado = concelhos[['MunicipalityCode', 'Municipality', 'Region']].drop_duplicates().copy()
dfConcelhosTratado.to_csv('concelhosTratados.csv', index=False)


## 3. Data Profiling (ADICIONAR MAIS DETALHES)

- Número de entradas
- Tipos de dados
- Percentagem de valores nulos
- Cardinalidade de colunas relevantes


In [38]:
def DataProfiling(dataframe: pd.DataFrame):
    # Data profiling completo para todas as colunas
    data_profiling = pd.DataFrame({
        "Coluna": dataframe.columns,
        "Tipo de Dado": dataframe.dtypes.values,
        "Valores Não Nulos": dataframe.notnull().sum().values,
        "Valores Nulos": dataframe.isnull().sum().values,
        "% Nulos": (dataframe.isnull().mean() * 100).round(2).values,
        "Valores Únicos": dataframe.nunique().values
    })
    display(data_profiling)  

### Areas_Freg_Conc_Dist_Pais_CAOP2019.xls



In [39]:
DataProfiling(concelhos)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,MunicipalityCode,object,308,0,0.0,308
1,Municipality,object,308,0,0.0,307
2,Region,object,308,0,0.0,25
3,MunicipalityNorm,object,308,0,0.0,307


⚙️ **Seleção de Colunas Relevantes — `Areas_Concelhos_CAOP2019`**

A tabela contém um total de **14 colunas**, mas apenas **3 foram consideradas essenciais** para os objetivos deste projeto:

- **`DICO`** — Utilizado como **identificador único do concelho** (chave primária).
- **`CONCELHO_DSG`** — Representa o **nome oficial do concelho**.
- **`NUTSIII_DSG`** — Refere-se à **região NUTS III**, fundamental para análises geográficas agregadas.
  
As restantes colunas, como área, perímetro e altitudes, foram excluídas por não serem relevantes para a análise urbana e socioeconómica focada neste estudo.


---

### densidadealojamentosm2.json



In [40]:
DataProfiling(densidade_aloj)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,geocod,object,3439,0,0.0,3439
1,geodsg,object,3439,0,0.0,3075
2,ind_string,object,3439,0,0.0,1656
3,valor,object,3439,0,0.0,1656


densidadelojamentosm2.json

| Coluna       | Tipo de Dado | Valores Não Nulos | Valores Nulos | % Nulos | Ação Planeada                     |
|--------------|--------------|-------------------|----------------|---------|-----------------------------------|
| `geocod`     | object       | 344               | 0              | 0.00%   | Manter                            |
| `geodsg`     | object       | 344               | 0              | 0.00%   | Usar como chave                   |
| `ind_string` | object       | 344               | 0              | 0.00%   | Ignorar                           |
| `valor`      | object       | 344               | 0              | 0.00%   | Converter para float              |


---

### DensidadePopulacional.json



In [41]:
DataProfiling(densidade_pop)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,geocod,object,347,0,0.0,347
1,geodsg,object,347,0,0.0,340
2,ind_string,object,347,0,0.0,316
3,valor,object,347,0,0.0,316


densidadePopulacional.json

| Coluna       | Tipo de Dado | Valores Não Nulos | Valores Nulos | % Nulos | Ação Planeada                     |
|--------------|--------------|-------------------|----------------|---------|-----------------------------------|
| `geocod`     | object       | 344               | 0              | 0.00%   | Manter                            |
| `geodsg`     | object       | 344               | 0              | 0.00%   | Usar como chave                   |
| `ind_string` | object       | 344               | 0              | 0.00%   | Ignorar                           |
| `valor`      | object       | 344               | 0              | 0.00%   | Converter para float              |


---

### portugal_listings.csv

In [42]:
DataProfiling(listings)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,Price,float64,135236,300,0.22,4754
1,District,object,135536,0,0.00,27
2,City,object,135536,0,0.00,275
3,Town,object,135534,2,0.00,2263
4,Unnamed: 4,float64,0,135536,100.00,0
5,Unnamed: 5,float64,0,135536,100.00,0
6,Unnamed: 6,float64,0,135536,100.00,0
7,Unnamed: 7,float64,0,135536,100.00,0
8,Unnamed: 8,float64,0,135536,100.00,0
9,Unnamed: 9,float64,0,135536,100.00,0


portugal_listings.csv

| Coluna                  | Tipo de Dado | Valores Não Nulos | Valores Nulos | % Nulos | Ação Planeada               |
|-------------------------|--------------|-------------------|----------------|---------|-----------------------------|
| `Price`                 | float64      | 135236            | 300            | 0.22%   | Remover nulos               |
| `District`              | object       | 135536            | 0              | 0.00%   | Manter                      |
| `City`                  | object       | 135536            | 0              | 0.00%   | Manter                      |
| `Town`                  | object       | 135534            | 2              | 0.001%  | remover                     |
| `Unnamed: 4` a `Unnamed: 13` | float64 | 0              | 135536         | 100%    | Remover todas               |


---

### rendasm2.json


In [43]:
DataProfiling(rendas)

,Coluna,Tipo de Dado,Valores Não Nulos,Valores Nulos,% Nulos,Valores Únicos
0,geocod,object,638,0,0.00,638
1,geodsg,object,638,0,0.00,616
2,ind_string,object,638,0,0.00,343
3,valor,object,431,207,32.45,341
4,sinal_conv,object,207,431,67.55,2
5,sinal_conv_desc,object,207,431,67.55,2


rendasm2.json

| Coluna            | Tipo de Dado | Valores Não Nulos | Valores Nulos | % Nulos | Ação Planeada                                 |
|-------------------|--------------|-------------------|----------------|---------|-----------------------------------------------|
| `geocod`          | object       | 638               | 0              | 0.00%   | Manter                                        |
| `geodsg`          | object       | 638               | 0              | 0.00%   | Usar como chave para junção                   |
| `ind_string`      | object       | 638               | 0              | 0.00%   | Ignorar                                       |
| `valor`           | object       | 431               | 207            | 32.45%  | Eliminar ou ignorar entradas nulas            |
| `sinal_conv`      | object       | 207               | 431            | 67.55%  | Apenas explicativo (motivo da ausência de valor) |
| `sinal_conv_desc` | object       | 207               | 431            | 67.55%  | Ignorar ou mover para anexo                   |


---

### Conclusões do Data Profiling

Após a leitura e análise dos datasets, foi possível identificar a seguinte situação:

- Os dados extraídos dos ficheiros `.json` foram corretamente convertidos em tabelas pandas a partir da chave `"Dados"`.
- As colunas `valor`, apesar de estarem no formato `object`, podem ser convertidas para `float` após a substituição da vírgula pelo ponto decimal.
- No dataset das rendas (`rendasm2.json`), foi detetada uma percentagem significativa de valores nulos (~32%) na coluna `valor`, que deverão ser removidos ou imputados.
- Todos os datasets possuem a coluna `geodsg`, que será usada como **chave principal de junção**.
- A coluna `geocod` foi mantida por agora, uma vez que poderá ser útil para cruzamentos futuros com dados geográficos oficiais.
- Os campos como `ind_string`, `sinal_conv` e `sinal_conv_desc` foram considerados auxiliares e não serão usados na análise principal.

**Próximos passos:**
- Conversão das colunas `valor` para `float`.
- Remoção dos registos com valores nulos nas variáveis principais.
- Normalização de nomes para futura integração entre datasets.


<hr/>

### Data Cleaning & Preparation (REVER)

Após a fase de data profiling, foi identificado que os datasets apresentam estruturas heterogéneas e inconsistências nos tipos de dados e formatação. Esta etapa tem como objetivo preparar os dados para integração e análise, assegurando a sua consistência e limpeza.

#### Principais problemas detetados:

- As colunas com valores numéricos estão no formato `string`;
- Alguns campos contêm separadores decimais diferentes, `"."` e `","`
- Existem registos com valores nulos, especialmente em colunas importantes `Price` e `valor`;
- Nomes (`District`, `Town`, `City`) contêm acentos, letras maiúsculas/minúsculas.
- **(PODE HAVER MAIS)**

#### Objetivos desta fase:

1. **Converter colunas com números para `float`**;
2. **Remover registos com valores nulos ou inválidos nas variáveis principais**;
3. **Normalizar nomes de localidades** para permitir junções consistentes entre datasets:
   - Remoção de acentos (se aplicável);
   - Conversão para minúsculas;
   - Eliminação de espaços antes/depois do texto;
4. **Criar colunas normalizadas (`*_norm`)** que servirão como chaves comuns para integração.

Com estas operações, os dados ficarão prontos para a fase seguinte de **integração de esquemas e resolução de identidade**.


In [23]:
!pip install unidecode


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
# Normalizar nomes
def normalizar(texto):
    return unidecode.unidecode(str(texto).strip().lower())

In [ ]:
import pandas as pd
import unidecode

file_path = 'dados/portugal_listings.csv'
df = pd.read_csv(file_path)

# Verificar colunas existentes
print("Columns in the DataFrame:", df.columns)

# Remover colunas 'Unnamed'
df = df.drop(columns=[col for col in df.columns if "Unnamed" in col])

# Converter 'Price' para float (números)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')  # erros → NaN

# Verificar valores nulos após conversão
print("Nº de valores nulos em 'Price':", df['Price'].isna().sum())

# Filtrar linhas com preço válido (> 0)
df_filtered = df[df['Price'].notna() & (df['Price'] > 0)].copy()

df_filtered['District_norm'] = df_filtered['District'].apply(normalizar)
df_filtered['City_norm'] = df_filtered['City'].apply(normalizar)
df_filtered['Town_norm'] = df_filtered['Town'].apply(normalizar)

print(df_filtered.head())
print("Número total de registos limpos:", len(df_filtered))


Columns in the DataFrame: Index(['Price', 'District', 'City', 'Town', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13'],
      dtype='object')
Nº de valores nulos em 'Price': 300
      Price   District                  City  \
0  780000.0  Vila Real              Valpaços   
1  223000.0       Faro  São Brás de Alportel   
2  228000.0       Faro  São Brás de Alportel   
3  250000.0       Faro  São Brás de Alportel   
4  250000.0       Faro  São Brás de Alportel   

                               Town District_norm             City_norm  \
0  Carrazedo de Montenegro e Curros     vila real              valpacos   
1              São Brás de Alportel          faro  sao bras de alportel   
2              São Brás de Alportel          faro  sao bras de alportel   
3              São Brás de Alportel          faro  sao bras de alportel   
4              São Brás de Alportel          faro

In [24]:
!pip install xlrd


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [46]:
import pandas as pd

# Caminho para o ficheiro
file_path = 'dados/Areas_Freg_Conc_Dist_Pais_CAOP2019.xls'

# Ler apenas a sheet dos conselhos (municípios)
df_concelhos = pd.read_excel(
    file_path,
    sheet_name='Areas_Concelhos_CAOP2019',
    dtype={'DICO': str, 'NUTSIII_COD': str}
)


# Verificar colunas disponíveis
df_concelhos = df_concelhos[['DICO', 'NUTSIII_DSG', 'NUTSIII_COD', 'DISTRITO_ILHA_DSG', 'CONCELHO_DSG']].copy()
df_concelhos = df_concelhos.rename(columns={
    'CONCELHO_DSG': 'municipio',
    'NUTSIII_DSG': 'regiao',
    'NUTSIII_COD': 'codigo_regiao',
    'DISTRITO_ILHA_DSG': 'distrito',
    'DICO': 'codigo_municipio'
})
print("Colunas:", df_concelhos.columns)

df_concelhos['municipio_norm'] = df_concelhos['municipio'].apply(normalizar)
df_concelhos['distrito_norm'] = df_concelhos['distrito'].apply(normalizar)
df_concelhos['regiao_norm'] = df_concelhos['regiao'].apply(normalizar)

df_concelhos['geocod'] = (
    df_concelhos['codigo_regiao'].astype(str).str.zfill(3) +
    df_concelhos['codigo_municipio'].astype(str).str.zfill(2)
)

# Visualizar as primeiras linhas
print(df_concelhos.head())

Colunas: Index(['codigo_municipio', 'regiao', 'codigo_regiao', 'distrito', 'municipio'], dtype='object')
  codigo_municipio                       regiao codigo_regiao distrito  \
0             0101             REGIÃO DE AVEIRO           16D   AVEIRO   
1             0102             REGIÃO DE AVEIRO           16D   AVEIRO   
2             0103             REGIÃO DE AVEIRO           16D   AVEIRO   
3             0104  ÁREA METROPOLITANA DO PORTO           11A   AVEIRO   
4             0105             REGIÃO DE AVEIRO           16D   AVEIRO   

            municipio      municipio_norm distrito_norm  \
0              ÁGUEDA              agueda        aveiro   
1  ALBERGARIA-A-VELHA  albergaria-a-velha        aveiro   
2              ANADIA              anadia        aveiro   
3              AROUCA              arouca        aveiro   
4              AVEIRO              aveiro        aveiro   

                   regiao_norm   geocod  
0             regiao de aveiro  16D0101  
1          

In [20]:
import pandas as pd
import unidecode

file_path = 'dados/rendasm2.json'
df = pd.read_json(file_path)

# Verificar colunas existentes
print("Columns in the DataFrame:", df.columns)

# Remover colunas 'Unnamed'
df = df.drop(columns=[col for col in df.columns if "Unnamed" in col])

# Converter 'Price' para float (números)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')  # erros → NaN

# Verificar valores nulos após conversão
print("Nº de valores nulos em 'Price':", df['Price'].isna().sum())

# Filtrar linhas com preço válido (> 0)
df_filtered = df[df['Price'].notna() & (df['Price'] > 0)].copy()

# Normalizar nomes
def normalizar(texto):
    return unidecode.unidecode(str(texto).strip().lower())

df_filtered['District_norm'] = df_filtered['District'].apply(normalizar)
df_filtered['City_norm'] = df_filtered['City'].apply(normalizar)
df_filtered['Town_norm'] = df_filtered['Town'].apply(normalizar)

print(df_filtered.head())
print("Número total de registos limpos:", len(df_filtered))


Columns in the DataFrame: Index(['IndicadorCod', 'IndicadorDsg', 'MetaInfUrl', 'DataExtracao',
       'DataUltimoAtualizacao', 'UltimoPref', 'Dados', 'Sucesso'],
      dtype='object')


KeyError: 'Price'

In [1]:
import xml.etree.ElementTree as ET

tree = ET.parse('concelhos.up.xml')  # Parse XML file
root = tree.getroot()  # Get root element

objects_data = []
for object_element in root.findall("Object"):
    object_dict = {}
    for property_element in object_element.findall("Property"):
        name = property_element.get("Name")
        value = property_element.text
        object_dict[name] = value
    objects_data.append(object_dict)

df = pd.DataFrame(objects_data)
    
print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'concelhos.up.xml'

In [63]:
import json
import pandas as pd

# Load the JSON file
file_path = 'dados/densidadealojamentosm2.json'  # Replace with the actual file path
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract the 2021 data
dados_2021 = data[0]['Dados']['2021']

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(dados_2021)

# Print the column names to verify
print("Columns in the DataFrame:", df.columns)

# Rename columns
df = df.rename(columns={
    'geocod': 'RegionCode',
    'geodsg': 'RegionName',
    'ind_string': 'DensityString',
    'valor': 'DensityValue'  # Ensure this matches the actual column name
})

# Convert DensityValue to numeric
df['DensityValue'] = pd.to_numeric(df['DensityValue'], errors='coerce')

# Handle missing values (e.g., replace "-" with NaN)
df['DensityValue'] = df['DensityValue'].replace('-', None)

# Filter out rows with missing or non-applicable data
df_filtered = df[df['DensityValue'].notna()]

# Display the cleaned DataFrame
print(df_filtered.head())

Columns in the DataFrame: Index(['geocod', 'geodsg', 'ind_string', 'valor'], dtype='object')
  RegionCode                                         RegionName DensityString  \
0     070513                                Torre de Coelheiros           2,1   
1     070601                                            Cabrela           2,2   
2     031003                                     Campo do Gerês           2,3   
3     120302                                       Aldeia Velha           2,4   
4     070525  União das freguesias de Nossa Senhora da Toure...           2,5   

   DensityValue  
0           2.1  
1           2.2  
2           2.3  
3           2.4  
4           2.5  


In [57]:
import json
import pandas as pd

# Load the JSON file
file_path = 'dados/densidadePopulacional.json'  # Replace with the actual file path
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract the 2022 data
dados_2022 = data[0]['Dados']['2022']

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(dados_2022)

# Print the column names to verify
print("Columns in the DataFrame:", df.columns)

# Rename columns
df = df.rename(columns={
    'geocod': 'RegionCode',
    'geodsg': 'RegionName',
    'ind_string': 'DensityString',
    'valor': 'DensityValue'  # Ensure this matches the actual column name
})

# Convert DensityValue to numeric
df['DensityValue'] = pd.to_numeric(df['DensityValue'], errors='coerce')

# Handle missing values (e.g., replace "-" with NaN)
df['DensityValue'] = df['DensityValue'].replace('-', None)

# Filter out rows with missing or non-applicable data
df_filtered = df[df['DensityValue'].notna()]

# Display the cleaned DataFrame
print(df_filtered.head())

Columns in the DataFrame: Index(['geocod', 'geodsg', 'ind_string', 'valor'], dtype='object')
  RegionCode     RegionName DensityString  DensityValue
0    1500802       Alcoutim           4,3           4.3
1    1840209        Mértola           4,8           4.8
2    16H0505  Idanha-a-Nova           5,9           5.9
3    1861203           Avis           6,2           6.2
4    1861211       Monforte           7,1           7.1


In [53]:
import json
import pandas as pd

# Load the JSON file
file_path = 'dados/rendasm2.json'  # Replace with the actual file path
with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Extract the 2023 data
dados_2023 = data[0]['Dados']['2023']

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(dados_2023)

# Print the column names to verify
print("Columns in the DataFrame:", df.columns)

# Rename columns
df = df.rename(columns={
    'geocod': 'RegionCode',
    'geodsg': 'RegionName',
    'ind_string': 'RentString',
    'valor': 'RentValue',  # Ensure this matches the actual column name
    'sinal_conv': 'SignalCode',
    'sinal_conv_desc': 'SignalDescription'
})

# Convert RentValue to numeric
df['RentValue'] = pd.to_numeric(df['RentValue'], errors='coerce')

# Handle missing values (e.g., replace "-" with NaN)
df['RentValue'] = df['RentValue'].replace('-', None)

# Filter out rows with missing or non-applicable data
df_filtered = df[df['RentValue'].notna()]

# Display the cleaned DataFrame
print(df_filtered.head())

Columns in the DataFrame: Index(['geocod', 'geodsg', 'ind_string', 'valor', 'sinal_conv',
       'sinal_conv_desc'],
      dtype='object')
  RegionCode            RegionName RentString  RentValue SignalCode  \
0    11D0914  Vila Nova de Foz Côa       2,08       2.08        NaN   
1    11E0410             Vila Flor       2,29       2.29        NaN   
2    1960913              Trancoso       2,41       2.41        NaN   
3    1960501              Belmonte       2,61       2.61        NaN   
4    11C1302                 Baião       2,63       2.63        NaN   

  SignalDescription  
0               NaN  
1               NaN  
2               NaN  
3               NaN  
4               NaN  
